# ConidtionFuser

In [2]:
import sys
sys.path.insert(0, '../')
import torch
from audiocraft.modules.conditioners import ConditionFuser

/home/karlos/Documents/workspace/venv/music/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
fuser_args = {'cross_attention_pos_emb': False,
              'cross_attention_pos_emb_scale': 1,
              'sum': [],
              'prepend': [],
              'cross': ['description'],
              'input_interpolate': []}

# ClassifierFreeGuidanceDropout

In [4]:
import torch
from audiocraft.modules.conditioners import ClassifierFreeGuidanceDropout, ConditioningAttributes

In [5]:
t = torch.rand(size = (1, 4, 10))

In [6]:
conditioning_attribute = ConditioningAttributes(text = {"description": "Here I am now"})
conditioning_attribute

ConditioningAttributes(text={'description': 'Here I am now'}, wav={}, joint_embed={})

In [8]:
ClassifierFreeGuidanceDropout(p=0.0)([conditioning_attribute])

[ConditioningAttributes(text={'description': 'Here I am now'}, wav={}, joint_embed={})]

# DelayedPatternProvider

In [9]:
from audiocraft.modules.codebooks_patterns import DelayedPatternProvider

In [13]:
pattern_provider = DelayedPatternProvider(4, delays=[0,1,2,3])

In [18]:
pattern = pattern_provider.get_pattern(15)

In [19]:
t = torch.rand(size = (1, 4, 10))

In [20]:
pattern.build_pattern_sequence(t, -1)

(tensor([[[-1.0000,  0.9081,  0.6864,  0.5895,  0.6139,  0.0775,  0.5167,
            0.2959,  0.1331,  0.2415,  0.4306, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000,  0.9293,  0.5303,  0.9221,  0.9487,  0.4307,
            0.7470,  0.5274,  0.5737,  0.9933,  0.4489, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  0.1292,  0.6397,  0.5820,  0.2968,
            0.9788,  0.1210,  0.7964,  0.2011,  0.7083,  0.7013, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000,  0.2545,  0.9201,  0.4730,
            0.3478,  0.6736,  0.9236,  0.3878,  0.1821,  0.0045,  0.8292,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000]]]),
 tensor([[40,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 40, 40, 40, 40, 40, 40, 40,
          40],
         [40, 40, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 40, 40, 40, 40, 40, 40

In [21]:
pattern.get_first_step_with_timesteps(0)

1

In [22]:
pattern.get_first_step_with_timesteps(1)

2

In [23]:
pattern.get_first_step_with_timesteps(2)

3

In [31]:
tt = torch.rand(2, 2, 2)
tt.split(1, dim=0)[0].shape

torch.Size([1, 2, 2])

# Dataset, DataLoaders

In [1]:
import torch
from audiocraft.solvers import builders
from audiocraft.environment import AudioCraftEnvironment
from audiocraft.utils.utils import get_loader

from omegaconf import OmegaConf
import os
os.environ["AUDIOCRAFT_CONFIG"] = "../config/teams/default.yaml"

/home/karlos/.pyenv/versions/3.9.18/envs/music/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class AttrDict(dict):
    """ Dictionary subclass whose entries can be accessed by attributes (as well
        as normally).

    >>> obj = AttrDict()
    >>> obj['test'] = 'hi'
    >>> print obj.test
    hi
    >>> del obj.test
    >>> obj.test = 'bye'
    >>> print obj['test']
    bye
    >>> print len(obj)
    1
    >>> obj.clear()
    >>> print len(obj)
    0
    """
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

    @classmethod
    def from_nested_dicts(cls, data):
        """ Construct nested AttrDicts from nested dictionaries. """
        if not isinstance(data, dict):
            return data
        else:
            return cls({key: cls.from_nested_dicts(data[key]) for key in data})



In [3]:
cfg_raw = {'sample_rate': 32000,
       'seed': 2036,
       'max_sample_rate': 32000,
       'max_channels': 1,
        'channels': 1,
       'dataset': {'batch_size': 1,
                   'num_workers': 10,
                   'segment_duration': 30,
                   'num_samples': None,
                   'return_info': True,
                   'shuffle': False,
                   'sample_on_duration': False,
                   'sample_on_weight': False,
                   'min_segment_ratio': 0.8,
                   'train': {'num_samples': 1000000, 'shuffle': True},
                   'valid': {'num_samples': 10000},
                   'evaluate': {'num_samples': 10000},
                   'generate': {'num_samples': 50, 'return_info': True, 'batch_size': 1}},
       'execute_only': None,
       'datasource': {'max_sample_rate': 44100, 
                      'max_channels': 2, 
                      'train': 'egs/example',
                      'valid': 'egs/example', 
                      'evaluate': 'egs/example', 
                      'generate': 'egs/example'},
       'optim': {
               # 'epochs': 500,
                 'updates_per_epoch': 2000,
                 # 'lr': 1,
                 # 'optimizer': 'dadam',
                 # 'adam': {'betas': [0.9, 0.95],
                 #          'weight_decay': 0.1,
                 #          'eps': 1e-08},
                 # 'ema': {'use': True,
                 #         'updates': 10,
                 #         'device': 'cpu',
                 #         'decay': 0.99},
                 # 'max_norm': 1.0,
                 # 'eager_sync': True
                },
           'return_info': True
    }



In [4]:
# cfg = AttrDict.from_nested_dicts(cfg_raw)
cfg = OmegaConf.create(cfg_raw)

In [5]:
dataloaders = builders.get_audio_datasets(cfg, dataset_type=builders.DatasetType.MUSIC)

/home/karlos/.pyenv/versions/3.9.18/envs/music/lib/python3.9/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f9980419b20>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x7f9812edc460>,
 'evaluate': <torch.utils.data.dataloader.DataLoader at 0x7f9812edc520>,
 'generate': <torch.utils.data.dataloader.DataLoader at 0x7f9812edc6a0>}

In [7]:
batch = next(iter(dataloaders['train']))

In [8]:
len(batch)

2

In [9]:
music, segment_with_attributes = batch

In [10]:
len(music), len(music[0][0]), len(music[0][0])//cfg.sample_rate

(1, 960000, 30)

In [11]:
segment_with_attributes[0]

MusicInfo(meta=AudioMeta(path='/home/karlos/Documents/workspace/projects/music/audiocraft/NEW_REFORMATTED/dataset/example/electro_1.mp3', duration=15.024, sample_rate=48000, amplitude=None, weight=None, info_path=None), seek_time=0.0, n_frames=480000, total_frames=960000, sample_rate=32000, channels=1, audio_tokens=None, title='Enracinement', artist='Voyager I', key=None, bpm=None, genre='electronic', moods=['uplifting', 'motivational'], keywords=['bright', 'pulsing', 'cool'], description='A cool song from Voyager.', name='electro_1', instrument='mix', self_wav=WavCondition(wav=tensor([[[0.0338, 0.0341, 0.0290,  ..., 0.0000, 0.0000, 0.0000]]]), length=tensor([480000]), sample_rate=[32000], path=['/home/karlos/Documents/workspace/projects/music/audiocraft/NEW_REFORMATTED/dataset/example/electro_1.mp3'], seek_time=[0.0]), joint_embed={})

In [12]:
len(dataloaders['train'])

1000000

In [13]:
from audiocraft import data

In [14]:
def dict_from_config(cfg) -> dict:
    """Convenience function to map an omegaconf configuration to a dictionary.

    Args:
        cfg (omegaconf.DictConfig): Original configuration to map to dict.
    Returns:
        dict: Config as dictionary object.
    """
    dct = OmegaConf.to_container(cfg, resolve=True)
    assert isinstance(dct, dict)
    return dct

In [15]:
path = 'egs/example'
dataset_cfg = dict_from_config(cfg.dataset)
splits_cfg: dict = {}
splits_cfg['train'] = dataset_cfg.pop('train')
splits_cfg['valid'] = dataset_cfg.pop('valid')
splits_cfg['evaluate'] = dataset_cfg.pop('evaluate')
splits_cfg['generate'] = dataset_cfg.pop('generate')
execute_only_stage = cfg.get('execute_only', None)

split_cfg = splits_cfg['train']
split_kwargs = {k: v for k, v in split_cfg.items()} #KM: 'train': {'num_samples': 1000000, 'shuffle': True, 'shuffle_seed': 0, 'permutation_on_files': False, 'merge_text_p': 0.25, 'drop_desc_p': 0.5, 'drop_other_p': 0.5}, 'valid': {'num_samples': 10000}, 'evaluate': {'num_samples': 10000}, 'generate': {'num_samples': 50, 'return_info': True}
kwargs = {**dataset_cfg, **split_kwargs}  # split kwargs overrides default dataset_cfg


return_info = kwargs.pop('return_info')
batch_size = kwargs.pop('batch_size', None)
num_workers = kwargs.pop('num_workers')

In [16]:
ds = data.music_dataset.MusicDataset.from_meta(path, **kwargs)

In [17]:
len(ds)

1000000

In [18]:
1000000*30*32000

960000000000

In [19]:
batch

(tensor([[[0.0338, 0.0341, 0.0290,  ..., 0.0000, 0.0000, 0.0000]]]),
 [MusicInfo(meta=AudioMeta(path='/home/karlos/Documents/workspace/projects/music/audiocraft/NEW_REFORMATTED/dataset/example/electro_1.mp3', duration=15.024, sample_rate=48000, amplitude=None, weight=None, info_path=None), seek_time=0.0, n_frames=480000, total_frames=960000, sample_rate=32000, channels=1, audio_tokens=None, title='Enracinement', artist='Voyager I', key=None, bpm=None, genre='electronic', moods=['uplifting', 'motivational'], keywords=['bright', 'pulsing', 'cool'], description='A cool song from Voyager.', name='electro_1', instrument='mix', self_wav=WavCondition(wav=tensor([[[0.0338, 0.0341, 0.0290,  ..., 0.0000, 0.0000, 0.0000]]]), length=tensor([480000]), sample_rate=[32000], path=['/home/karlos/Documents/workspace/projects/music/audiocraft/NEW_REFORMATTED/dataset/example/electro_1.mp3'], seek_time=[0.0]), joint_embed={})])